# RAG with NVIDIA NIM Microservices

Welcome to this lab! In this notebook, you'll learn how to build a production-grade Retrieval-Augmented Generation (RAG) pipeline using NVIDIA NIM microservices.

## What You'll Learn
- **NVIDIA NIMs**: How to integrate hosted microservices for Embeddings, Reranking, and LLM generation.
- **RAG Architecture**: Building a complete pipeline from ingestion to generation.
- **Vector Stores**: Using FAISS for efficient similarity search.
- **Guardrails**: Implementing topic control to keep the AI focused.

## Technologies Used
| Component | Technology | Purpose |
|-----------|------------|---------|
| **Embeddings** | `nvidia/nv-embed-v1` | High-performance text embeddings |
| **LLM** | `meta/llama-3.2-1b-instruct` | Efficient instruction-tuned generation |
| **Reranker** | `nvidia/llama-3.2-nv-rerankqa-1b-v2` | Improving retrieval relevance |
| **Guardrails** | `llama-3.1-nemoguard-8b-topic-control` | Input/Output safety and steering |
| **Orchestration** | LangChain | Pipeline management |
| **Vector DB** | FAISS | Similarity search engine |

## Lab Flow
1.  **Setup**: Install dependencies and configure API keys.
2.  **Initialization**: Connect to NVIDIA NIM clients.
3.  **Ingestion**: Load PDFs, split text, and create vector embeddings.
4.  **Basic RAG**: Retrieve documents and generate answers.
5.  **Guardrails**: Apply topic control to restrict the assistant's scope.
6.  **Advanced RAG**: Add reranking to improve answer quality.

---

## System Architecture

The diagram below illustrates the RAG pipeline we will build.

<img src="./docs/RAG_WITH_NIMS.png" alt="RAG Architecture" width="800"/>

### Architecture Description

The system is designed in two main stages:

#### 1. Data Ingestion (Preprocessing)
- **Document Loader**: Reads PDF files from the `./pdf` directory.
- **Text Splitter**: Breaks documents into manageable chunks (800 chars) with overlap to preserve context.
- **Embedding Model (`nv-embed-v1`)**: Converts text chunks into dense vector representations.
- **Vector Store (FAISS)**: Indexes these vectors for fast similarity search.

#### 2. Inference Pipeline (User Query)
- **Topic Control (`nemoguard-8b`)**: First, the user's query is checked against a specific prompt (e.g., "Only answer HR questions"). If off-topic, the system refuses politely.
- **Retriever**: If on-topic, the system embeds the query and searches the FAISS index for the top 20 most similar chunks.
- **Reranker (`nv-rerankqa-1b`)**: These 20 chunks are re-scored by a cross-encoder model to find the truly relevant ones, filtering out noise.
- **LLM Generator (`llama-3.2-1b`)**: The top reranked documents are combined with the user's query into a prompt. The LLM generates a factual answer with citations.

---


### Scenario: InnovateSphere Corporation

We'll work with documents from InnovateSphere, a fictional company with three departments:
- **HR**: Employee policies, leave entitlements, code of conduct
- **Marketing**: Brand guidelines, campaigns, social media policies  
- **Sales**: Product catalogs, sales reports, team directories


### NVIDIA NIMs Used

| Model | Purpose |
|-------|----------|
| `nvidia/nv-embed-v1` | Generate 4096-dim embeddings for semantic search |
| `meta/llama-3.2-1b-instruct` | Generate natural language responses |
| `nvidia/llama-3.2-nv-rerankqa-1b-v2` | Rerank documents by relevance |
| `nvidia/llama-3.1-nemoguard-8b-topic-control` | Enforce topic boundaries (guardrails) |

---


## 1. Install Dependencies

Install required packages for RAG implementation:
- **LangChain ecosystem**: Framework for LLM applications
- **FAISS**: Vector database for similarity search
- **NVIDIA AI Endpoints**: Integration with NVIDIA NIMs
- **PyPDF**: PDF document parsing


In [ ]:
# Install compatible versions
%pip install -q \
    pypdf \
    faiss-cpu \
    "langchain>=0.3,<0.4" \
    "langchain-core>=0.3,<0.4" \
    "langchain-text-splitters>=0.3,<0.4" \
    langchain-community \
    langchain_nvidia_ai_endpoints \
    openai \
    numpy==1.26.4

print("Dependencies installed successfully.")

## 2. Configure NVIDIA API Key

To authenticate with the NVIDIA API Catalog, you need to set your personal API key. This key allows you to access the hosted models via LangChain.

If you haven't already generated your key, follow the step-by-step guide below.

> **Important:** Never share your API key publicly or commit it to source control.


### How to Generate Your NVIDIA API Key

Follow these steps to generate your API key:

---

#### Step 1: Log in to NVIDIA Build

Go to the [NVIDIA API Keys page](https://build.nvidia.com/settings/api-keys) and log in using your NVIDIA account credentials.

<img src="./docs/key_guide/login.png" alt="Step 1: Log in to NVIDIA Build" width="900"/>

---

#### Step 2: Navigate to API Key Settings

Once logged in, click on the **"API Keys"** tab in the sidebar or top navigation menu.

<img src="./docs/key_guide/menu.png" alt="Step 2: API Key Menu" width="900"/>

---

#### Step 3: Click "Generate API Key"

Click the **"Generate API Key"** button to start creating a new key.

<img src="./docs/key_guide/generate.png" alt="Step 3: Generate API Key" width="900"/>

---

#### Step 4: Fill Out the API Key Form

You'll be prompted to fill in some details like a name and expiry time for the key. Complete the form and click **Generate key**.

<img src="./docs/key_guide/form.png" alt="Step 4: Fill API Key Form" width="900"/>

---

#### Step 5: Copy and Store Your Key Securely

After the key is generated, **copy it immediately** and store it somewhere safe. You **won't be able to view it again**.

<img src="./docs/key_guide/copy.png" alt="Step 5: Copy API Key" width="900"/>

---

### Set Your API Key

Now that you have your API key, paste it in the code cell below:


In [ ]:
API_KEY = "API KEY HERE" # Paste your actual API key here

## 3. Initialize NVIDIA NIMs Clients

Initialize four microservices:

**Embeddings (`nv-embed-v1`)**: Converts text to dense vectors for semantic search. Based on Mistral-7B with Latent-Attention pooling.

**LLM (`llama-3.2-1b-instruct`)**: 1B parameter instruction-tuned model for answer generation. Configured with low temperature (0.2) for factual responses.

**Reranker (`llama-3.2-nv-rerankqa-1b-v2`)**: Re-scores retrieved documents for improved relevance. Supports up to 8192 tokens.

**Topic Control (`llama-3.1-nemoguard-8b-topic-control`)**: Classifies queries as on-topic/off-topic to enforce conversational boundaries.


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
from openai import OpenAI


# Initialize the Embeddings client to get vector representations of documents and queries
embedding_client = NVIDIAEmbeddings(
    model="nvidia/nv-embed-v1",
    api_key=API_KEY,
    truncate="NONE",
)

# Initialize Chat client for LLM generation
gpt_client = ChatNVIDIA(
    model="meta/llama-3.2-1b-instruct",
    api_key=API_KEY,
    temperature=0.2,
    top_p=0.5,
)

# Initialize the reranker client for passage reranking
reranker_client = NVIDIARerank(
    model="nvidia/llama-3.2-nv-rerankqa-1b-v2",
    api_key=API_KEY,
)

# Initialize Topic Control client for guardrails
topic_control_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY,
)

print("NVIDIA clients initialized successfully.")

## 4. Load PDF Documents

Load all PDFs from the `./pdf` directory recursively. The loader extracts text and preserves metadata (source file, page numbers).


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./pdf")
data = loader.load()

## 5. Chunk Documents

Split documents into 800-character chunks with 60-character overlap. This balances:
- **Chunk size**: Small enough for focused retrieval, large enough for context
- **Overlap**: Prevents information loss at chunk boundaries

The splitter intelligently preserves semantic coherence by splitting on paragraphs, then sentences, then words.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=60,
)
documents = text_splitter.split_documents(data)

## 6. Create Vector Store

Generate embeddings for all chunks and build a FAISS index for fast similarity search.

In [ ]:
import numpy as np
from langchain.vectorstores import FAISS
docsearch = FAISS.from_documents(documents, embedding=embedding_client)
docsearch.save_local(folder_path="./embeddings")

## 7. Load Pre-computed Embeddings

Load the existing FAISS index from disk. This contains embeddings for all document chunks.

> **Security**: `allow_dangerous_deserialization=True` is required for pickle-based serialization. Only load from trusted sources.


In [ ]:
from langchain.vectorstores import FAISS
docsearch= FAISS.load_local(
    "./embeddings" , embedding_client, allow_dangerous_deserialization=True
)

## 8. Configure Retriever

Create a retriever that returns the top 20 most semantically similar chunks for each query. The retriever:
1. Embeds the query using `nv-embed-v1`
2. Performs cosine similarity search in FAISS
3. Returns the k=20 nearest document chunks


In [ ]:
retriever = docsearch.as_retriever(search_kwargs={"k": 20}) 

## 9. Define Topic Control Prompts (Guardrails)

Define domain-specific prompts that restrict AI assistants to their designated topics:

- **HR Assistant**: Only answers HR-related questions (policies, leave, conduct)
- **Marketing Assistant**: Only answers marketing questions (campaigns, branding)
- **Sales Assistant**: Only answers sales questions (products, reports)

These guardrails prevent assistants from answering out-of-scope questions, ensuring information security and role-based access control.


In [ ]:
HR_TOPIC_CONTROL_PROMPT = (
    "You are an HR assistant for InnovateSphere. Only answer questions about HR policies, employee handbook, leave entitlements, Code of Conduct, diversity, workplace guidelines, or performance review procedures. Do not answer any queries about marketing, sales, products, financials, or campaigns. If the query is outside HR topics, respond that you cannot provide such information."
)
MARKETING_TOPIC_CONTROL_PROMPT = (
    "You are a marketing assistant for InnovateSphere. Only answer questions about InnovateSphere’s marketing campaigns, brand guidelines, messaging, target markets, and social media policies. Do not address HR policies, sales results, product information, or internal staff matters. Respond with a polite rejection if the question is outside marketing topics."
)
SALES_TOPIC_CONTROL_PROMPT = (
    "You are a sales assistant for InnovateSphere. Provide information only about sales reports, product catalog, sales team directory, and quarterly performance. Never answer questions relating to HR, marketing, or internal policies. If the query is not about sales, reject it as outside sales scope."
)

## 10. Import RAG Pipeline

Import the custom `RAGPipeline` class that orchestrates:
- Topic control (optional guardrails)
- Document retrieval
- Reranking (optional quality improvement)
- LLM answer generation
- Source citation and formatting


In [ ]:
import RAGPipeline

## 11. Initialize RAG Pipeline

Create a pipeline instance with all NVIDIA NIMs clients. The pipeline provides a unified interface:

```python
pipeline.query(
    query="Your question",
    enable_topic_control=True,  # Optional: Enable guardrails
    enable_rerank=True          # Optional: Enable reranking
)
```


In [ ]:
topic_control_model = "nvidia/llama-3.1-nemoguard-8b-topic-control"

pipeline = RAGPipeline.RAGPipeline(
    retriever=retriever,
    gpt_client=gpt_client,
    reranker_client=reranker_client,
    topic_control_client=topic_control_client,
    topic_control_model=topic_control_model,
)

---

# Part 1: Basic RAG Queries

Test the RAG system without guardrails or reranking. The pipeline will:
1. Retrieve top 20 relevant chunks
2. Generate an answer using the LLM
3. Display source documents with citations


### Query 1: HR Content

Ask about InnovateSphere's professional integrity statement from HR documents.


In [ ]:
pipeline.query("What is the professional integrity statement of InnovateSphere?")

### Query 2: Sales Content

Retrieve Q1 sales performance figures from sales reports.


In [ ]:
pipeline.query("Show the Q1 sales performance figures.")

### Query 3: Marketing Content

Query brand identity guidelines from marketing documents.


In [ ]:
pipeline.query("Describe InnovateSphere's brand identity guidelines.")

---

# Part 2: RAG with Topic Control (Guardrails)

Demonstrate how guardrails enforce domain boundaries. The topic control model classifies queries as "on-topic" or "off-topic" based on the configured prompt.


### Demo 1: HR Assistant - On-Topic Query

Configure as HR assistant and ask an HR-related question. Expected: Answer provided.


In [ ]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("How many days of annual leave do employees receive?", enable_topic_control=True)

### Demo 2: HR Assistant - Off-Topic Query

Ask the HR assistant a sales question. Expected: Polite refusal explaining the question is out of scope.


In [ ]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("Provide the product catalog details.", enable_topic_control=True)

### Demo 3: Sales Assistant - Same Query

Ask the same product catalog question to a Sales assistant. Expected: Answer provided (now on-topic).


In [ ]:
pipeline.topic_control_prompt = SALES_TOPIC_CONTROL_PROMPT
pipeline.query("Provide the product catalog details.", enable_topic_control=True)

### Demo 4: Marketing Assistant

Test marketing domain guardrails with a campaign-related query.


In [ ]:
pipeline.topic_control_prompt = MARKETING_TOPIC_CONTROL_PROMPT
pipeline.query("What is the target persona for the SynergyHub campaign?", enable_topic_control=True)

---

# Part 3: RAG with Reranking

Enable reranking to improve retrieval quality. The reranker:
1. Takes the initial 20 retrieved documents
2. Compares each document to the query using a cross-encoder
3. Re-scores and reorders documents by relevance
4. Provides relevance scores in the output


### Query with Reranking

Ask about performance review criteria with reranking enabled. Notice the relevance scores in the output.


In [ ]:
pipeline.query("Detail performance review criteria for staff.", enable_rerank=True)

---

# Part 4: Full Pipeline (Topic Control + Reranking)

Combine both features for production-grade RAG:
- **Topic Control**: Ensures queries are within scope
- **Reranking**: Maximizes answer quality


### Full Pipeline Demo 1: HR Domain

HR assistant with both guardrails and reranking enabled.


In [ ]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("Explain the diversity commitment at InnovateSphere.", enable_topic_control=True, enable_rerank=True)

### Full Pipeline Demo 2: Marketing Domain

Marketing assistant with full features enabled.


In [ ]:
pipeline.topic_control_prompt = MARKETING_TOPIC_CONTROL_PROMPT
pipeline.query("What are the brand usage rules?", enable_topic_control=True, enable_rerank=True)

### Full Pipeline Demo 3: Sales Domain

Sales assistant with topic control and reranking for optimal performance.


In [ ]:
pipeline.topic_control_prompt = SALES_TOPIC_CONTROL_PROMPT
pipeline.query("Provide the product catalog details", enable_topic_control=True, enable_rerank=True)

---

## Summary

You've successfully built a production-grade RAG system with:

✅ **Semantic Search**: FAISS vector database with NVIDIA embeddings  
✅ **LLM Generation**: Context-aware answers with citations  
✅ **Guardrails**: Topic control for domain-specific assistants  
✅ **Quality Optimization**: Reranking for improved relevance  

### Next Steps

- Experiment with your own queries
- Compare results with/without reranking
- Create custom topic control prompts
- Adjust retrieval parameters (k, temperature, top_p)

### Resources

- [NVIDIA API Catalog](https://build.nvidia.com/)
- [NeMo Guardrails](https://github.com/NVIDIA/NeMo-Guardrails)
- [LangChain Docs](https://python.langchain.com/)
